In [1]:
import cv2
import numpy as np

def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size = (512,512))
    image = image.astype('float32')
    image /= 255.0  # Normalize pixel values to [0, 1]
    return image


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_model(input_shape=(224, 224, 3)):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False)
    base_model.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)  # Assuming binary classification (vehicle or non-vehicle)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [ ]:
model = build_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming you have train and validation data
train_data = content/Data/train
val_data = content/Data/validate

model.fit(train_data, validation_data=val_data, epochs=10)

In [ ]:
test_data = ...

loss, accuracy = model.evaluate(test_data)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


In [ ]:
def detect_vehicle(image_path, threshold=0.5):
    image = preprocess_image(image_path)
    prediction = model.predict(np.expand_dims(image, axis=0))
    if prediction[0][0] >= threshold:
        return "Vehicle detected!"
    else:
        return "No vehicle detected."

image_path = 'test_image.jpg'
print(detect_vehicle(image_path))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming you have two folders: 'train' and 'validation', each containing subfolders for each class
train_data_dir = '/content/Data/train'
validation_data_dir = '/content/Data/validate'

# Define data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,  # Rescale pixel values to [0, 1]
    shear_range=0.2,   # Randomly applies shearing transformations
    zoom_range=0.2,    # Randomly applies zooming transformations
    horizontal_flip=True)  # Randomly flips images horizontally

val_datagen = ImageDataGenerator(rescale=1. / 255)  # Only rescale validation images

# Define batch size
batch_size = 32

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Specify target size
    batch_size=batch_size,
    class_mode='binary')  # Assuming binary classification

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),  # Specify target size
    batch_size=batch_size,
    class_mode='binary')  # Assuming binary classification

# Build the model
model = build_model()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)
